In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import math
from scipy.stats import norm

In [5]:
S=80
K=80
T=0.5
rf=0.02
Volatilité=0.18

In [6]:
def black_scholes(option_type, S, K, r, sigma, T):
    """
    Calcule le prix d'une option européenne selon le modèle de Black-Scholes.

    Arguments :
    - option_type : 'call' pour un call, 'put' pour un put
    - S : prix actuel de l'actif sous-jacent
    - K : prix d'exercice de l'option
    - r : taux d'intérêt sans risque (en décimal)
    - sigma : volatilité du sous-jacent (en décimal)
    - T : temps restant jusqu'à l'expiration de l'option (en années)

    Retourne :
    - Le prix de l'option
    """
    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Le type d'option doit être 'call' ou 'put'.")

    return option_price


In [7]:
print(black_scholes(option_type='call', S=80, K=80, r=0.02, sigma=0.18, T=0.5))
print(black_scholes(option_type='put', S=80, K=80, r=0.02, sigma=0.18, T=0.5))

4.449678784149157
3.653665484082609


In [8]:
def crr_binary_option(option_type, S, K, r, sigma, T, N):
    """
    Calcule le prix d'une option binaire (options à deux états) selon le modèle de Cox-Ross-Rubinstein (CRR).

    Arguments :
    - option_type : 'call' pour un call, 'put' pour un put
    - S : prix actuel de l'actif sous-jacent
    - K : prix d'exercice de l'option
    - r : taux d'intérêt sans risque par période (en décimal)
    - sigma : volatilité du sous-jacent par période (en décimal)
    - T : temps restant jusqu'à l'expiration de l'option (en années)
    - N : nombre de périodes

    Retourne :
    - Le prix de l'option binaire
    """
    delta_t = T / N
    u = math.exp(sigma * math.sqrt(delta_t))
    d = 1 / u
    p = (math.exp(r * delta_t) - d) / (u - d)

    # Construction du tableau des prix de l'actif sous-jacent à chaque nœud
    prices = [[S]]

    for i in range(1, N + 1):
        node_prices = []
        prev_prices = prices[i - 1]
        for j in range(i + 1):
            if j == 0:
                price = prev_prices[j] * u
            else:
                price = prev_prices[j - 1] * d
            node_prices.append(price)
        prices.append(node_prices)

    # Calcul des valeurs d'option à l'expiration
    option_values = [max(0, price-K) if option_type == 'call' else  max(0, K-price) for price in prices[N]]

    # Remontée du tableau des valeurs d'option
    for i in range(N - 1, -1, -1):
        option_values = [(p * option_values[j] + (1 - p) * option_values[j + 1]) * math.exp(-r * delta_t) for j in range(i + 1)]

    return option_values[0]


In [9]:
print(crr_binary_option(option_type='call', S=80, K=80, r=0.02, sigma=0.18,T=0.5,N=3))
print(crr_binary_option(option_type='put', S=80, K=80, r=0.02, sigma=0.18,T=0.5,N=3))

4.79021461034006
3.99420131027353


In [10]:
def find_N(option_type, S, K, r, sigma, T, epsilon):
    N = 999
    while True:
        crr_price = crr_binary_option(option_type, S, K, r, sigma, T, N)
        black_scholes_price = black_scholes(option_type, S, K, r, sigma, T)
        diff = abs(crr_price - black_scholes_price)
        if diff < epsilon:
            break
        N += 1
    return N

In [11]:
epsilon = 0.1
N = find_N(option_type='call', S=80, K=80, r=0.02, sigma=0.18, T=0.5, epsilon=0.001)
print("Le nombre de périodes N pour une différence inférieure à 0.001 est :", N)

Le nombre de périodes N pour une différence inférieure à 0.001 est : 999


In [12]:
"""S = cours du sous-jacent
   K = prix exercice
   T = maturité en années
   sigma= volatilité annuelle du sous-jacent
   m = valeur moyenne du choc sur le cours du sous-jacent
   v = écart-type du choc
   lamda = nombre de chocs par année
   V_BS = valeur option par la formule de Black and Scholes
"""

import numpy as np

def prolongement_BS(option_type, S, K, r, sigma, T, m, v, lamda, n):
   
   sigma_n = np.sqrt(sigma**2 + 0*(v**2/T))

   r_n = r - lamda*(m-1) + 0*np.log(m)/T
   log_sum = sum(np.log(x) for x in range(1,n+1))
   option_price = 0
   for i in range(1,n+1):
      sigma_n = np.sqrt(sigma**2 + i*(v**2/T))
      r_n = r - lamda*(m-1) + i*np.log(m)/T
      if option_type == 'call':
        option_price = option_price + np.exp(-m*lamda*T + i*np.log(m*lamda*T)-log_sum)*black_scholes('call', S, K, r_n, sigma_n, T)
      elif option_type == 'put':
        option_price = option_price + np.exp(-m*lamda*T + i*np.log(m*lamda*T)-log_sum)*black_scholes('put', S, K, r_n, sigma_n, T)
      else:
        raise ValueError("Le type d'option doit être 'call' ou 'put'.")
   return option_price

In [13]:
n=10
log_sum = sum(np.log(x) for x in range(1,n+1))
log_sum


15.104412573075518

In [30]:
S=100
K=100
T=1
r=0.02
sigma=0.2
m=0
v=0.03
lamda=1
n=50
print(prolongement_BS(option_type='call', S=80, K=80, r=0.02, sigma=0.18, T=0.5, m=1, v=0.03, lamda=1, n=50))

9.30260150007248e-65


In [35]:
import numpy as np

def prolongement_BS2(option_type, S, K, r, sigma, T, m, v, lamda, n):
   
   sigma_n = np.sqrt(sigma**2 + 1*(v**2/T))

   r_n = r - lamda*(m-1) + 1*np.log(m)/T
   log_sum = sum(np.log(x) for x in range(1,n+1))
   option_price = 0
   for i in range(n+1):
      sigma_n = np.sqrt(sigma**2 + i*(v**2/T))
      r_n = r - lamda*(m-1) + i*np.log(m)/T
      option_price = option_price + np.exp(-m*lamda*T + i*np.log(m*lamda*T)-log_sum)*black_scholes(option_type, S, K, r_n, sigma_n, T)
      
   return option_price

In [58]:
print(prolongement_BS2(option_type='call', S=100, K=100, r=0.02, sigma=0.2, T=0.5, m=100, v=0.3, lamda=1, n=50))

5.747449625019393


In [37]:
import numpy as np

def prolongement_BS3(option_type, S, K, r, sigma, T, m, v, lamda, n):
   option_price = 0
   if m==0:
      option_price = option_price
   else:
      sigma_n = np.sqrt(sigma**2 + 1*(v**2/T))
      r_n = r - lamda*(m-1) + 1*np.log(m)/T
      log_sum = sum(np.log(x) for x in range(1,n+1))
   
      for i in range(n+1):
         sigma_n = np.sqrt(sigma**2 + i*(v**2/T))
         r_n = r - lamda*(m-1) + i*np.log(m)/T
         option_price = option_price + np.exp(-m*lamda*T + i*np.log(m*lamda*T)-log_sum)*black_scholes(option_type, S, K, r_n, sigma_n, T)   
   return option_price

In [48]:
print(prolongement_BS3(option_type='call', S=100, K=100, r=0.02, sigma=0.2, T=1, m=0.00001, v=0.3, lamda=1, n=50))
print(black_scholes(option_type='call', S=100, K=100, r=0.02, sigma=0.2, T=1))

2.1022986263576264e-63
8.916037278572539


In [17]:
def facto(n):
    if (n==0):
        factoriel=1
    else:
        factoriel=n*facto(n-1)
    return factoriel

      

In [32]:
facto(5)

120